### An ensemble of Neural networks to classify the Fashion MNIST images

In [1]:
from keras.models import load_model
from keras.datasets import fashion_mnist


import numpy as np
import matplotlib.pyplot

/home/pier/anaconda3/envs/tensorflow/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


### Load the fasion MNIST data

In [2]:
# Load the training and testing data
(X_train, Y_train), (X_test, Y_test) = fashion_mnist.load_data()
# Display purpose:
X_train_orig = X_train
X_test_orig = X_test

In [3]:
from keras import backend as K
img_rows, img_cols = 28, 28

if K.image_data_format() == 'channels_first':
    shape_ord = (1, img_rows, img_cols)
else:  # channel_last
    shape_ord = (img_rows, img_cols, 1)
    

In [4]:
X_train = X_train.reshape((X_train.shape[0],) + shape_ord)
X_test = X_test.reshape((X_test.shape[0],) + shape_ord)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

X_train /= 255
X_test /= 255

## Load the NN models that are already trained

In [5]:
cnn_model = load_model('cnn.h5')
mlff_bp_model = load_model('mlff_bp_model.h5')
rnn_bi_gru_model = load_model('bidirectional_gru.h5')
rnn_bi_gru_model.name = "rnn"


In [6]:
def get_classification_accuracy_ensemble(models_with_accuracy, test_images, test_labels):
    """
    Get the classification accuracy of the models in the ensemble
    """
    models_with_accuracy = [(model, accuracy * 100) for (model, accuracy) in models_with_accuracy if accuracy <= 1]
    
    num_test_images = len(test_images)
    num_test_labels = len(test_labels)
    
    if num_test_images != num_test_labels:
        raise ValueError("The number of test images does not equal the number of test labels.")

    prediction_results = []
    for model, accuracy in models_with_accuracy:
        # print("Accuracy: " + str(accuracy))
        
        # Get arrays (of size 10) of predictions for each test image
        if model.name=="rnn":
            original_predictions = model.predict(np.squeeze(test_images)) # rnn doesn't take in dimension of 1 
        else:
            original_predictions = model.predict(test_images)
        # print(original_predictions)
        
        # multiply each element of predictions for each test image with the accuracy
        weighted_predictions = [accuracy * predictions_for_test_image for predictions_for_test_image in original_predictions]
        # print("Weighted predictions...")
        # print(weighted_predictions)
        prediction_results.append(weighted_predictions)
        
    ensembles_predictions = []
    for itr in range(num_test_images):
        prediction_sum = np.asarray([0.0]*10)
        for prediction_result in prediction_results:
            prediction_sum += (prediction_result[itr])
        # print(prediction_sum)
        ensembles_predictions.append(prediction_sum.argmax(-1))
        
    correct_classifications = 0
    for idx, prediction in enumerate(ensembles_predictions):
        if prediction == test_labels[idx]:
            correct_classifications += 1
            
    print("{0} classified correctly out of {1}. Classification accuracy: {2}".format(correct_classifications, str(num_test_images), correct_classifications/num_test_images))
    return ensembles_predictions

### Get the classification accuracy of the ensemble

In [7]:
slice = 10000
models_with_accuracy = [(cnn_model, 0.9), (rnn_bi_gru_model, 0.89), (mlff_bp_model, 0.88)]
test_data = X_test[:slice]
test_labels = Y_test[:slice]

ensemble_predictions = get_classification_accuracy_ensemble(models_with_accuracy, test_data, test_labels)

9088 classified correctly out of 10000. Classification accuracy: 0.9088


### Generate the confusion matrix for the predictions

In [8]:
from sklearn.metrics import confusion_matrix

item_label_mapping = ["T-shirt/Top", "Trouser", "Pullover", "Dress", "Coat", "Sandal", "Shirt", "Sneaker", "Bag", "Ankle Boot"]

print(item_label_mapping)
print("Actuals ->")
c = confusion_matrix(Y_test, ensemble_predictions)


print(c)

['T-shirt/Top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle Boot']
Actuals ->
[[880   0  14  21   2   1  73   0   9   0]
 [  1 971   0  23   2   0   2   0   1   0]
 [ 14   0 859  13  67   0  46   0   1   0]
 [ 17   2   8 934  20   0  14   0   5   0]
 [  1   1  64  34 846   0  53   0   1   0]
 [  0   0   0   0   0 981   0  14   1   4]
 [125   2  69  26  54   0 712   0  12   0]
 [  0   0   0   0   0  10   0 981   0   9]
 [  1   1   2   5   2   3   0   3 983   0]
 [  1   0   0   0   0   6   0  52   0 941]]
